# Demo SingleRun
This notebook shows a single run consisting of the following four phases:
* sail empty
* loading
* sail full
* unloading

In [15]:
import datetime, time
import simpy

import shapely.geometry

import pandas as pd
import openclsim.core as core
import openclsim.model as model

import openclsim.plugins as plugin

# setup environment
simulation_start  = datetime.datetime(2009,1,1)
my_env = simpy.Environment(initial_time=simulation_start.timestamp())
registry = {}
keep_resources = {}

import numpy as np

In [16]:
Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasContainer,
        core.HasResource,
    ),
    {},
)

TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,
        core.Log,
        core.ContainerDependentMovable,
        core.Processor,
        core.HasResource,
    ),
    {},
)

TestMoveActivity =  type(
    "TestMoveActivity",
    (
        plugin.HasWeatherPluginActivity,
        model.MoveActivity,  # the order is critical!
    ),
    {},
)


TestShiftActivity =  type(
    "TestShiftActivity",
    (
        plugin.HasWeatherPluginActivity,
        model.ShiftAmountActivity,  # the order is critical!
    ),
    {},
)

In [17]:
location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)  # lon, lat
location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)  # lon, lat

data_from_site = {
    "env": my_env,
    "name": "Winlocatie",
    "geometry": location_from_site,
    "capacity": 40,
    "level": 40,
}


data_to_site = {
    "env": my_env,
    "name": "Dumplocatie",
    "geometry": location_to_site,
    "capacity": 40,
    "level": 0,
}

from_site = Site(**data_from_site)
to_site = Site(**data_to_site)

In [18]:
data_hopper = {
    "env": my_env,
    "name": "Hopper 01",
    "geometry": location_from_site,
    "capacity": 4,
    "compute_v":lambda x: 10,
}

hopper = TransportProcessingResource(**data_hopper)

In [19]:
metocean_df = pd.read_csv("data/unit_test_weather.csv")
metocean_df = metocean_df.set_index(pd.to_datetime(metocean_df["Time"], dayfirst=True))
metocean_df = metocean_df.sort_index()

metocean_df["Hs [m]"] = 4 + \
                        1.5*np.sin(metocean_df['  Hour']/24 * 8 *np.pi) + \
                        1.5*np.sin(metocean_df['  Hour']/24 * 6 *np.pi)

metocean_df = metocean_df.set_index(pd.to_datetime(metocean_df["Time"], dayfirst=True))
metocean_df = metocean_df.sort_index()
metocean_df["ts"] = metocean_df.index.values.astype(float) / 1_000_000_000

In [20]:
sailing_crit = plugin.WeatherCriterion(**{
    "name":"sailing_crit",
    "condition":"Hs [m]",
    "maximum":6,
    "window_length":3600
})

loading_crit = plugin.WeatherCriterion(**{
    "name":"loading_crit",
    "condition":"Hs [m]",
    "maximum":4.5,
    "window_length":3600
})

In [21]:
single_run = [
    TestMoveActivity(**{
        "env": my_env,
        "name": "Soil movement",
        "registry": registry,
        "mover": hopper,
        "destination": from_site,
        "metocean_criteria":sailing_crit,
        "metocean_df": metocean_df,
        "postpone_start": True,
    }),
    TestShiftActivity(**{
        "env": my_env,
        "name": "Transfer MP",
        "registry": registry,
        "processor": hopper,
        "origin": from_site,
        "destination": hopper,
        "amount": 4,
        "duration": 3600,
        "postpone_start": True,
        "metocean_criteria":loading_crit,
        "metocean_df": metocean_df,
    }),
    TestMoveActivity(**{
        "env": my_env,
        "name": "Soil movement",
        "registry": registry,
        "mover": hopper,
        "destination": to_site,
        "metocean_criteria":sailing_crit,
        "metocean_df": metocean_df,
        "postpone_start": True,
    }),
    TestShiftActivity(**{
        "env": my_env,
        "name": "Transfer TP",
        "registry": registry,
        "processor": hopper,
        "origin": hopper,
        "destination": to_site,
        "amount": 4,
        "duration": 3600,
        "postpone_start": True,
        "metocean_criteria":loading_crit,
        "metocean_df": metocean_df,
    })
]

activity = model.SequentialActivity(**{
    "env": my_env,
    "name": "Single run process",
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff60",
    "registry": registry,
    "sub_processes": single_run,
    "postpone_start": True,
})

expr = [{"type":"container", "concept": to_site, "state":"full"}]
while_data = {
    "env": my_env,
    "name": "while",
    "registry": registry,
    "sub_process": activity,
    "condition_event": expr,
    "postpone_start": False,
}
while_activity = model.WhileActivity(**while_data)


In [22]:
my_env.run()

In [23]:
pd.DataFrame(single_run[0].log).head(50)

,Message,Timestamp,Value,Geometry,ActivityID,ActivityState
0,move activity Soil movement of Hopper 01 to Wi...,2009-01-01 00:00:00.000000,-1,None,eb19f928-35a3-4397-b87c-25f284fd1607,WAIT_START
1,move activity Soil movement of Hopper 01 to Wi...,2010-01-01 03:00:00.000000,-1,None,eb19f928-35a3-4397-b87c-25f284fd1607,WAIT_STOP
2,move activity Soil movement of Hopper 01 to Wi...,2010-01-01 03:00:00.000000,-1,POINT (4.18055556 52.18664444),eb19f928-35a3-4397-b87c-25f284fd1607,START
3,move activity Soil movement of Hopper 01 to Wi...,2010-01-01 03:00:00.000000,-1,POINT (4.18055556 52.18664444),eb19f928-35a3-4397-b87c-25f284fd1607,STOP
4,move activity Soil movement of Hopper 01 to Wi...,2010-01-01 06:15:42.824591,-1,None,eb19f928-35a3-4397-b87c-25f284fd1607,WAIT_START
5,move activity Soil movement of Hopper 01 to Wi...,2010-01-01 06:15:42.824591,-1,None,eb19f928-35a3-4397-b87c-25f284fd1607,WAIT_STOP
6,move activity Soil movement of Hopper 01 to Wi...,2010-01-01 06:15:42.824591,-1,POINT (4.25222222 52.11428333),eb19f928-35a3-4397-b87c-25f284fd1607,START
7,move activity Soil movement of Hopper 01 to Wi...,2010-01-01 06:31:25.649182,-1,POINT (4.18055556 52.18664444),eb19f928-35a3-4397-b87c-25f284fd1607,STOP
8,move activity Soil movement of Hopper 01 to Wi...,2010-01-01 11:00:00.000000,-1,None,eb19f928-35a3-4397-b87c-25f284fd1607,WAIT_START
9,move activity Soil movement of Hopper 01 to Wi...,2010-01-01 11:00:00.000000,-1,None,eb19f928-35a3-4397-b87c-25f284fd1607,WAIT_STOP


In [24]:
metocean_df

,Year,Month,Day,Hour,Hour.1,Time,U10 [m/s],Hs [m],ts
Time,,,,,,,,,
2010-01-01 00:00:00,2010,1,1,0,1,01-01-2010 00:00,10,4.000000,1.262304e+09
2010-01-01 01:00:00,2010,1,1,1,2,01-01-2010 01:00,10,6.359698,1.262308e+09
2010-01-01 02:00:00,2010,1,1,2,3,01-01-2010 02:00,10,6.799038,1.262311e+09
2010-01-01 03:00:00,2010,1,1,3,4,01-01-2010 03:00,10,5.060660,1.262315e+09
2010-01-01 04:00:00,2010,1,1,4,5,01-01-2010 04:00,10,2.700962,1.262318e+09
...,...,...,...,...,...,...,...,...,...
2010-01-08 19:00:00,2010,1,8,19,20,08-01-2010 19:00,10,6.359698,1.262977e+09
2010-01-08 20:00:00,2010,1,8,20,21,08-01-2010 20:00,10,5.299038,1.262981e+09
2010-01-08 21:00:00,2010,1,8,21,22,08-01-2010 21:00,10,2.939340,1.262984e+09
